# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "mtecbo"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-15 10:26:50,819 - INFO - Iniciando Sessão Spark.


# Parâmetros para leitura do arquivo

In [3]:
schema = T.StructType([
    T.StructField("CODIGO", T.StringType(), True),
    T.StructField("TITULO", T.StringType(), True)
])

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("mte/cbo")
local = f"s3a://landing/mte/cbo/{ano_mes}/CBO2002 - Ocupacao.csv"

In [5]:
config_adicional = {
    'header': True,
    'sep': ';',
    'encoding': 'iso-8859-1'
}

# Leitura do arquivo

In [6]:
df_compliance_cbo_ocupacao = extrator_bronze.criar_view_temporaria_arquivo(local, 'compliance_cbo_ocupacao',schema,**config_adicional)

2024-10-15 10:26:58,632 - INFO - Lista de yaml carregada com sucesso
2024-10-15 10:27:05,750 - INFO - Criando view temporaria.


In [7]:
sql_query = """
    SELECT
        *,
        SUBSTRING(codigo, 1, 4) AS cd_familia
    FROM
        compliance_cbo_ocupacao
"""

In [8]:
df_compliance_cbo_ocupacao = extrator_bronze.carregar_dados_delta(sql_query)

2024-10-15 10:27:05,804 - INFO - Aguarde... Executando Query (Delta)
2024-10-15 10:27:05,806 - INFO - 
    SELECT
        *,
        SUBSTRING(codigo, 1, 4) AS cd_familia
    FROM
        compliance_cbo_ocupacao

2024-10-15 10:27:06,362 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [9]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/mte/cboocupacao'

In [10]:
extrator_bronze.salvar_delta(df_compliance_cbo_ocupacao, 'overwrite')

2024-10-15 10:27:06,431 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-15 10:27:28,353 - INFO - Dados persistidos com sucesso
2024-10-15 10:27:28,355 - INFO - s3a://bronze/mte/cboocupacao
2024-10-15 10:27:28,355 - INFO - Aguarde... Realizando optimize
2024-10-15 10:27:34,828 - INFO - Optimize executado com sucesso.
2024-10-15 10:27:34,829 - INFO - Aguarde... Realizando vacuum
2024-10-15 10:27:54,919 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [11]:
extrator_bronze.parar_sessao()

2024-10-15 10:27:55,218 - INFO - Sessão Spark finalizada.
